In [1]:
import pandas as pd
import re
from collections import Counter
from datetime import datetime
import calendar

In [3]:
files = ['checkyourfact.xlsx',
 'politifact.xlsx',
 'snopes.xlsx',
 'altnews.xlsx',
 'boomlive.xlsx',
 'opindia.xlsx']

In [4]:
def conv(s):
    s = s[1:-1]
    dic = {}

    positions=[match.end() for match in re.finditer(r'\b(?:positive|negative|neutral)\b,', s, flags=re.IGNORECASE)]
    for ss in [s[i:j] for i, j in zip([0] + positions, positions + [None])]:
        tmp = ss.split(':')
        try:
            dic[tmp[0]] = tmp[1].replace(",", "")
        except:
            continue

    return dic

def prepros(df):
    df.sentiments = df.sentiments.str.replace('[\n"\']', '', regex=True)
    df.sentiments = df.sentiments.str.replace(r'{\s+', '{', regex=True)
    df.sentiments = df.sentiments.str.replace(r'\s+}', '}', regex=True)
    df.sentiments = df.sentiments.str.replace(r':\s+', ':', regex=True)
    df.sentiments = df.sentiments.str.replace(r',\s+', ',', regex=True)
    df.sentiments = df.sentiments.apply(conv)

    return df

def map_to_root(df, mapping):
    top = list(mapping.index)
    top_dic = {}
    for dic in df.sentiments:
        for k in dic:
            #print(k)
            if k in top:
                #print(mapping.loc[k].loc['map'])
                try:
                    top_dic[mapping.loc[k].loc['map']] += [dic[k]]
                except:
                    #print(k)
                    #print(mapping.loc[k])
                    top_dic[mapping.loc[k].loc['map']] = [dic[k]]

    return top_dic

def sent_count(l):
    dic = {'positive': 0, 'negative': 0, 'neutral': 0}
    for i in l:
        try:
            dic[i] +=1
        except:
            continue

    return dic

def polarity_score(d):
    P = d['positive']
    N = d['negative']
    T = sum(d.values())

    try:
        return (P-N)/T, (P*0.0 + N*0.29412)/T #(P, N, T-P-N)
    except:
        return 0, 0

In [5]:
def Main(f, y, m):
    mapping = pd.read_excel('Top Topics/'+f)
    mapping = mapping[mapping.include==1]

    mapping.index = list(mapping.ent)

    df = pd.read_excel('Topic Sentiment Data/'+f)

    if(y!='all'):
        df = df[df.date_year==y]
        #df = df[df.date_month.isin(m)]

    df = prepros(df)

    top_senti_list = map_to_root(df, mapping)

    for k in top_senti_list:
        top_senti_list[k] = sent_count(top_senti_list[k])

    top_senti_list = dict(sorted(top_senti_list.items(), key=lambda x: x[1]['positive'] + x[1]['negative'] + x[1]['neutral'], reverse=True))

    return top_senti_list

In [6]:
for f in files:
    top_senti_all = dict(list(Main(f, 'all', 'all').items())[:5]).keys()
    print(f, "\n", top_senti_all)
    plot_data = pd.DataFrame(columns=['year']+list(top_senti_all))
    plot_dataE = pd.DataFrame(columns=['year']+list(top_senti_all))
    for y in ['all', 2018, 2019, 2020, 2021, 2022, 2023]:
        for m in range(1):#[calendar.month_name[i].capitalize() for i in range(1, 13)]:
            top_senti_list = Main(f, y, 'all')

            PS = {'year': y}
            PSE = {'year': y}
            for k in top_senti_all:
                try:
                    #PS[k] = top_senti_list[k]#polarity_score(top_senti_list[k])
                    PS[k], PSE[k] = polarity_score(top_senti_list[k])
                except:
                    PS[k], PSE[k] = 0, 0

            #print("year: ", y, end='\n')
            plot_data.loc[len(plot_data)] = PS
            plot_dataE.loc[len(plot_data)] = PSE

    ent_sort = list(plot_data.columns)[1:]
    ent_sort.sort(reverse=True)
    plot_data = plot_data[['year']+ent_sort]
    plot_dataE = plot_dataE[['year']+ent_sort]

    #plot_data.to_excel('GraphData/'+f, index=False)
    #plot_dataE.to_excel('GraphData/Err_'+f, index=False)

checkyourfact.xlsx 
 dict_keys(['donald trump', 'joe biden', 'democratic party', 'republican party', 'barack obama'])
politifact.xlsx 
 dict_keys(['donald trump', 'joe biden', 'democratic party', 'republican party', 'barack obama'])
snopes.xlsx 
 dict_keys(['donald trump', 'joe biden', 'democratic party', 'republican party', 'barack obama'])
altnews.xlsx 
 dict_keys(['bjp', 'narendra modi', 'congress', 'rahul gandhi', 'yogi adityanath'])
boomlive.xlsx 
 dict_keys(['bjp', 'narendra modi', 'congress', 'rahul gandhi', 'arvind kejriwal'])
opindia.xlsx 
 dict_keys(['bjp', 'congress', 'narendra modi', 'rahul gandhi', 'aap'])
